In [23]:
import os
for root, dirs, files in os.walk("/Users/wangqiuyi/Downloads/"):
    for name in files:
        if name == "chromedriver":
            print(os.path.join(root, name))


/Users/wangqiuyi/Downloads/chromedriver-mac-arm64/chromedriver


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ========== Initialization ==========
driver_path = "/Users/wangqiuyi/Downloads/chromedriver-mac-arm64/chromedriver"
options = Options()
# options.add_argument('--headless')  # Optional: run in headless mode
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# ========== Category links ==========
category_links = [
    ("Food Cupboard", "https://www.pnp.co.za/c/pnpbase?query=:relevance:allCategories:pnpbase:category:food-cupboard-423144840"),
    ("Personal Care & Hygiene", "https://www.pnp.co.za/c/pnpbase?query=:relevance:allCategories:pnpbase:category:personal-care-and-hygiene-423144840"),
    ("Liquor Store", "https://www.pnp.co.za/c/pnpbase?query=:relevance:allCategories:pnpbase:category:liquor-store-423144840"),
    # Add more categories if needed...
]

# ========== Product scraping function ==========
def scrape_products(category_name, category_url, max_items=30):
    print(f"Scraping: {category_name}")
    driver.get(category_url)
    time.sleep(5)

    # Scroll to trigger lazy loading
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)

    # Wait until product elements are loaded
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".product-grid-item[data-cnstrc-item-name]"))
        )
    except:
        print(f"[Warning] No items found for {category_name}")
        return pd.DataFrame()

    # Extract product elements
    items = driver.find_elements(By.CSS_SELECTOR, ".product-grid-item[data-cnstrc-item-name]")

    data = []
    for p in items[:max_items]:
        try:
            data.append({
                "Category": category_name,
                "ItemID": p.get_attribute("data-cnstrc-item-id"),
                "ItemName": p.get_attribute("data-cnstrc-item-name"),
                "Price": p.get_attribute("data-cnstrc-item-price"),
                "IsPromotion": "On%20Promotion" in category_url
            })
        except:
            continue

    return pd.DataFrame(data)

# ========== Main loop ==========
all_products = pd.DataFrame()
for name, url in category_links:
    df = scrape_products(name, url)
    all_products = pd.concat([all_products, df], ignore_index=True)

driver.quit()

# ========== Preview result ==========
print(all_products.head())
print(f"\nTotal products collected: {len(all_products)}")

# ========== Save to CSV if needed ==========
all_products.to_csv("sample_main.csv", index=False)


Scraping: Food Cupboard
Scraping: Personal Care & Hygiene
Scraping: Liquor Store
        Category                 ItemID  \
0  Food Cupboard  000000000000349246_CS   
1  Food Cupboard  000000000000349246_EA   
2  Food Cupboard  000000000000211805_EA   
3  Food Cupboard  000000000000515687_EA   
4  Food Cupboard  000000000000195758_EA   

                                     ItemName  Price  IsPromotion  
0              PnP UHT Full Cream Milk 6 x 1L  94.99        False  
1                  PnP UHT Full Cream Milk 1L  16.99        False  
2  Knorrox Chilli Beef Soya Mince in Bag 400g  32.99        False  
3           Knorr Vegetable Stock Pot 4 x 28g  36.99        False  
4                             Tastic Rice 2kg  34.99        False  

Total products collected: 90
